In [ ]:
import mediapipe as mp
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, accuracy_score, f1_score, recall_score, confusion_matrix, roc_curve, auc
from keras.utils.np_utils import to_categorical
import warnings
warnings.filterwarnings('ignore')

# Drawing helpers
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Load all sklearn models
with open("./model/all_sklearn.pkl", "rb") as f:
    sklearn_models = pickle.load(f)

# Load all deep learning models
with open("./model/all_dp.pkl", "rb") as f:
    dp_models = pickle.load(f)

# Load input scaler
with open("./model/input_scaler.pkl", "rb") as f:
    sc = pickle.load(f)

# Load dataset
def describe_dataset(dataset_path: str):
    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")
    return data

def round_up_metric_results(results) -> list:
    return list(map(lambda el: round(el, 3), results))

test_df = describe_dataset("./test.csv")

# Categorizing label
test_df.loc[test_df["label"] == "C", "label"] = 0
test_df.loc[test_df["label"] == "L", "label"] = 1

# Standard Scaling of features
test_x = test_df.drop("label", axis=1)
test_x = pd.DataFrame(sc.transform(test_x))
test_y = test_df["label"].astype('int')

# Converting prediction to categorical
test_y_cat = to_categorical(test_y)

# Deep Learning Model with Transfer Learning
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load MobileNet model without the top layer
base_model = MobileNet(weights='imagenet', include_top=False)
for layer in base_model.layers:
    layer.trainable = False

# Create the new model with added top layers
num_classes = test_y_cat.shape[1]  # Adjust based on your number of classes
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(test_x, test_y_cat, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
scores = model.evaluate(test_x, test_y_cat)
print(f"Accuracy: {scores[1]*100}")

# Visualization
plt.figure(figsize=(12, 5))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Accuracy')

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title('Loss')

plt.show()


## 1. Important Landmarks and Important functions

In [14]:
# Determine important landmarks for plank
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "RIGHT_ELBOW",
    "LEFT_ELBOW",
    "RIGHT_WRIST",
    "LEFT_WRIST",
    "LEFT_HIP",
    "RIGHT_HIP",
]

# Generate all columns of the data frame

HEADERS = ["label"] # Label column

for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

In [19]:
def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


# Remove duplicate rows (optional)
def remove_duplicate_rows(dataset_path: str):
    '''
    Remove duplicated data from the dataset then save it to another files
    '''
    
    df = pd.read_csv(dataset_path)
    df.drop_duplicates(keep="first", inplace=True)
    df.to_csv(f"cleaned_train.csv", sep=',', encoding='utf-8', index=False)


def round_up_metric_results(results) -> list:
    '''Round up metrics results such as precision score, recall score, ...'''
    return list(map(lambda el: round(el, 3), results))

## 2. Describe Dataset & Split Data

In [20]:
# load dataset
df = describe_dataset("./train.csv")

# Categorizing label
df.loc[df["label"] == "C", "label"] = 0
df.loc[df["label"] == "L", "label"] = 1

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v']
Number of rows: 15372 
Number of columns: 37

Labels: 
label
C    8238
L    7134
Name: count, dtype: int64

Missing values: False

Duplicate Rows : 0


In [21]:
with open("./model/input_scaler.pkl", "rb") as f:
    sc = pickle.load(f)

In [23]:
# Standard Scaling of features
x = df.drop("label", axis = 1)
x = pd.DataFrame(sc.transform(x))

y = df["label"]

# # Converting prediction to categorical
y_cat = to_categorical(y)

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2, random_state=1234)

## 3. Build Model

### 3.1. Set up

In [7]:
stop_early = EarlyStopping(monitor='val_loss', patience=3)

# Final Results
final_models = {}

In [27]:
def describe_model(model):
    '''
    Describe Model architecture
    '''
    print(f"Describe models architecture")
    for i, layer in enumerate(model.layers):
        number_of_units = layer.units if hasattr(layer, 'units') else 0

        if hasattr(layer, "activation"):
            print(f"Layer-{i + 1}: {number_of_units} units, func: ", layer.activation)
        else:
            print(f"Layer-{i + 1}: {number_of_units} units, func: None")


def get_best_model(tuner):
    '''
    Describe and return the best model found from keras tuner
    '''
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_model = tuner.hypermodel.build(best_hps)

    describe_model(best_model)

    print("\nOther params:")
    ignore_params = ["tuner", "activation", "layer", "epoch"]
    for param, value in best_hps.values.items():
        if not any(word in param for word in ignore_params):
            print(f"{param}: {value}")

    return best_model

### 3.2. Model with 3 layers

In [28]:
def model_3l_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [29]:
tuner_3l = kt.Hyperband(
    model_3l_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo',
)
tuner_3l.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early])

Trial 30 Complete [00h 00m 13s]
val_accuracy: 0.9980487823486328

Best val_accuracy So Far: 0.9983739852905273
Total elapsed time: 00h 02m 47s


In [30]:
model_3l = get_best_model(tuner_3l)
model_3l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x0000029C33F8F4C0>
Layer-2: 192 units, func:  <function tanh at 0x0000029C33F8F820>
Layer-3: 2 units, func:  <function softmax at 0x0000029C33F86A60>

Other params:
learning_rate: 0.001
Epoch 1/100
1230/1230 [==============================] - 4s 3ms/step - loss: 0.0587 - accuracy: 0.9817 - val_loss: 0.0174 - val_accuracy: 0.9935
Epoch 2/100
1230/1230 [==============================] - 3s 3ms/step - loss: 0.0225 - accuracy: 0.9941 - val_loss: 0.0153 - val_accuracy: 0.9954
Epoch 3/100
1230/1230 [==============================] - 3s 2ms/step - loss: 0.0191 - accuracy: 0.9941 - val_loss: 0.0077 - val_accuracy: 0.9980
Epoch 4/100
1230/1230 [==============================] - 3s 3ms/step - loss: 0.0174 - accuracy: 0.9948 - val_loss: 0.0250 - val_accuracy: 0.9932
Epoch 5/100
1230/1230 [==============================] - 3s 2ms/step - loss: 0.0146 - accuracy: 0.9959 - val_loss: 0.0093 - val_accuracy: 0.9974
Epoch 6/100
123

In [31]:
final_models["3_layers"] = model_3l

### 3.3. Model with 5 layers

In [35]:
def model_5l_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [36]:
tuner_5l = kt.Hyperband(
    model_5l_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_2'
)
tuner_5l.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early, TensorBoard("./keras_tuner_dir/logs")])

Trial 30 Complete [00h 00m 24s]
val_accuracy: 0.9967479705810547

Best val_accuracy So Far: 0.9980487823486328
Total elapsed time: 00h 05m 25s


In [25]:
model_5l = get_best_model(tuner_5l)
model_5l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x16bf85b80>
Layer-2: 160 units, func:  <function relu at 0x16bf85b80>
Layer-3: 352 units, func:  <function relu at 0x16bf85b80>
Layer-4: 64 units, func:  <function relu at 0x16bf85b80>
Layer-5: 2 units, func:  <function softmax at 0x16bf85160>

Other params:
learning_rate: 0.001
Epoch 1/100


2022-11-23 10:15:07.538823: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1230/1230 [==============================] - ETA: 0s - loss: 0.0494 - accuracy: 0.9848

2022-11-23 10:15:21.397335: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1230/1230 [==============================] - 16s 13ms/step - loss: 0.0494 - accuracy: 0.9848 - val_loss: 0.0152 - val_accuracy: 0.9958
Epoch 2/100
1230/1230 [==============================] - 16s 13ms/step - loss: 0.0238 - accuracy: 0.9932 - val_loss: 0.0145 - val_accuracy: 0.9954
Epoch 3/100
1230/1230 [==============================] - 16s 13ms/step - loss: 0.0193 - accuracy: 0.9947 - val_loss: 0.0146 - val_accuracy: 0.9971
Epoch 4/100
1230/1230 [==============================] - 16s 13ms/step - loss: 0.0169 - accuracy: 0.9950 - val_loss: 0.0140 - val_accuracy: 0.9964
Epoch 5/100
1230/1230 [==============================] - 15s 13ms/step - loss: 0.0160 - accuracy: 0.9960 - val_loss: 0.0154 - val_accuracy: 0.9964
Epoch 6/100
1230/1230 [==============================] - 16s 13ms/step - loss: 0.0135 - accuracy: 0.9963 - val_loss: 0.0126 - val_accuracy: 0.9961
Epoch 7/100
1230/1230 [==============================] - 16s 13ms/step - loss: 0.0126 - accuracy: 0.9960 - val_loss: 0.0098 - val_

In [ ]:
final_models["5_layers"] = model_5l

### 3.4. Model with 7 layers include Dropout

In [38]:
def model_7lD_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_dropout_1 = hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)
    hp_dropout_2 = hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_1))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_2))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [39]:
tuner_7lD = kt.Hyperband(
    model_7lD_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_3'
)
tuner_7lD.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early, TensorBoard("./keras_tuner_dir/logs")])

Trial 30 Complete [00h 00m 11s]
accuracy: 0.9899975657463074

Best accuracy So Far: 0.9975603818893433
Total elapsed time: 00h 05m 49s


In [40]:
model_7lD = get_best_model(tuner_7lD)
model_7lD.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x0000029C33F8F4C0>
Layer-2: 96 units, func:  <function relu at 0x0000029C33F8F4C0>
Layer-3: 0 units, func: None
Layer-4: 128 units, func:  <function relu at 0x0000029C33F8F4C0>
Layer-5: 0 units, func: None
Layer-6: 128 units, func:  <function relu at 0x0000029C33F8F4C0>
Layer-7: 2 units, func:  <function softmax at 0x0000029C33F86A60>

Other params:
dropout_1: 0.1
dropout_2: 0.1
learning_rate: 0.001
Epoch 1/100
1230/1230 [==============================] - 5s 3ms/step - loss: 0.0545 - accuracy: 0.9812 - val_loss: 0.0189 - val_accuracy: 0.9948
Epoch 2/100
1230/1230 [==============================] - 4s 3ms/step - loss: 0.0202 - accuracy: 0.9943 - val_loss: 0.0094 - val_accuracy: 0.9961
Epoch 3/100
1230/1230 [==============================] - 3s 2ms/step - loss: 0.0160 - accuracy: 0.9949 - val_loss: 0.0105 - val_accuracy: 0.9971
Epoch 4/100
1230/1230 [==============================] - 3s 2ms/step - loss: 0.0142 - ac

In [41]:
final_models["7_layers_with_dropout"] = model_7lD

### 3.5. Model with 7 layers

In [42]:
def model_7l_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_layer_4 = hp.Int('layer_4', min_value=32, max_value=512, step=32)
    hp_layer_5 = hp.Int('layer_5', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(units=hp_layer_4, activation=hp_activation))
    model.add(Dense(units=hp_layer_5, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [43]:
tuner_7l = kt.Hyperband(
    model_7l_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_6'
)
tuner_7l.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early, TensorBoard("./keras_tuner_dir/logs")])

Trial 23 Complete [00h 00m 15s]
val_accuracy: 0.9983739852905273

Best val_accuracy So Far: 0.9986991882324219
Total elapsed time: 00h 03m 53s

Search: Running Trial #24

Value             |Best Value So Far |Hyperparameter
tanh              |relu              |activation
256               |480               |layer_1
256               |192               |layer_2
352               |160               |layer_3
192               |288               |layer_4
352               |160               |layer_5
0.001             |0.001             |learning_rate
4                 |10                |tuner/epochs
0                 |4                 |tuner/initial_epoch
1                 |2                 |tuner/bracket
0                 |2                 |tuner/round

Epoch 1/4
345/385 [=========================>....] - ETA: 0s - loss: 0.0915 - accuracy: 0.9669

KeyboardInterrupt: 

In [ ]:
model_7l = get_best_model(tuner_7l)
model_7l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x13a058b80>
Layer-2: 192 units, func:  <function tanh at 0x13a058ee0>
Layer-3: 320 units, func:  <function tanh at 0x13a058ee0>
Layer-4: 448 units, func:  <function tanh at 0x13a058ee0>
Layer-5: 224 units, func:  <function tanh at 0x13a058ee0>
Layer-6: 448 units, func:  <function tanh at 0x13a058ee0>
Layer-7: 2 units, func:  <function softmax at 0x13a058160>

Other params:
learning_rate: 0.0001
Epoch 1/100
   1/1230 [..............................] - ETA: 8:04 - loss: 0.6561 - accuracy: 0.6000

2022-11-23 14:29:10.795739: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1230/1230 [==============================] - ETA: 0s - loss: 0.0720 - accuracy: 0.9748

2022-11-23 14:29:24.355056: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1230/1230 [==============================] - 16s 13ms/step - loss: 0.0720 - accuracy: 0.9748 - val_loss: 0.0219 - val_accuracy: 0.9941
Epoch 2/100
1230/1230 [==============================] - 15s 12ms/step - loss: 0.0319 - accuracy: 0.9910 - val_loss: 0.0353 - val_accuracy: 0.9893
Epoch 3/100
1230/1230 [==============================] - 15s 12ms/step - loss: 0.0262 - accuracy: 0.9927 - val_loss: 0.0149 - val_accuracy: 0.9958
Epoch 4/100
1230/1230 [==============================] - 15s 12ms/step - loss: 0.0221 - accuracy: 0.9936 - val_loss: 0.0125 - val_accuracy: 0.9964
Epoch 5/100
1230/1230 [==============================] - 15s 12ms/step - loss: 0.0195 - accuracy: 0.9943 - val_loss: 0.0171 - val_accuracy: 0.9951
Epoch 6/100
1230/1230 [==============================] - 16s 13ms/step - loss: 0.0186 - accuracy: 0.9942 - val_loss: 0.0128 - val_accuracy: 0.9971
Epoch 7/100
1230/1230 [==============================] - 16s 13ms/step - loss: 0.0170 - accuracy: 0.9951 - val_loss: 0.0132 - val_

In [ ]:
final_models["7_layers"] = model_7l

### 3.6. Describe final models

In [ ]:
for name, model in final_models.items():
    print(f"{name}: ", end="")
    describe_model(model)
    print()

3_layers: Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x17fe29b80>
Layer-2: 448 units, func:  <function tanh at 0x17fe29ee0>
Layer-3: 2 units, func:  <function softmax at 0x17fe29160>

5_layers: Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x17fe29b80>
Layer-2: 160 units, func:  <function relu at 0x17fe29b80>
Layer-3: 352 units, func:  <function relu at 0x17fe29b80>
Layer-4: 64 units, func:  <function relu at 0x17fe29b80>
Layer-5: 2 units, func:  <function softmax at 0x17fe29160>

7_layers_with_dropout: Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x17fe29b80>
Layer-2: 320 units, func:  <function relu at 0x17fe29b80>
Layer-3: 0 units, func: None
Layer-4: 96 units, func:  <function relu at 0x17fe29b80>
Layer-5: 0 units, func: None
Layer-6: 448 units, func:  <function relu at 0x17fe29b80>
Layer-7: 2 units, func:  <function softmax at 0x17fe29160>

7_layers: Describe models architecture
Layer-1: 36 units,

## 4. Model Evaluation

### 4.1. Train set evaluation

In [ ]:
train_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(x_test, verbose=False) 
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(y_test, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1])
    
    train_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

train_set_results.sort(key=lambda k: sum(k[3]), reverse=True)
pd.DataFrame(train_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,7_layers,"[0.998, 0.997]","[0.998, 0.997]","[0.998, 0.997]","[[1675, 4], [4, 1392]]"
1,3_layers,"[0.997, 0.996]","[0.997, 0.996]","[0.997, 0.996]","[[1674, 5], [5, 1391]]"
2,7_layers_with_dropout,"[0.998, 0.995]","[0.996, 0.997]","[0.997, 0.996]","[[1672, 7], [4, 1392]]"
3,5_layers,"[0.996, 0.996]","[0.996, 0.996]","[0.996, 0.996]","[[1673, 6], [6, 1390]]"


### 4.2. Test set evaluation

In [ ]:
# load dataset
test_df = describe_dataset("./test.csv")

# Categorizing label
test_df.loc[test_df["label"] == "C", "label"] = 0
test_df.loc[test_df["label"] == "L", "label"] = 1

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v']
Number of rows: 604 
Number of columns: 37

Labels: 
C    339
L    265
Name: label, dtype: int64

Missing values: False

Duplicate Rows : 0


In [ ]:
# Standard Scaling of features
test_x = test_df.drop("label", axis = 1)
test_x = pd.DataFrame(sc.transform(test_x))

test_y = test_df["label"]

# # Converting prediction to categorical
test_y_cat = to_categorical(test_y)

In [ ]:
test_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(test_x, verbose=False) 
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(test_y_cat, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1])
    
    test_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

test_set_results.sort(key=lambda k: k[1] + k[2] + k[3], reverse=True)
pd.DataFrame(test_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

2022-11-25 15:34:28.174069: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-25 15:34:28.287586: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-25 15:34:28.423087: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-25 15:34:28.546100: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,7_layers,"[0.944, 1.0]","[1.0, 0.925]","[0.971, 0.961]","[[339, 0], [20, 245]]"
1,5_layers,"[0.926, 1.0]","[1.0, 0.898]","[0.962, 0.946]","[[339, 0], [27, 238]]"
2,7_layers_with_dropout,"[0.909, 0.963]","[0.973, 0.875]","[0.94, 0.917]","[[330, 9], [33, 232]]"
3,3_layers,"[0.896, 0.983]","[0.988, 0.853]","[0.94, 0.913]","[[335, 4], [39, 226]]"


## 5. Dumped Model

In [ ]:
# Dump the best model to a pickle file
with open("./model/bicep_dp.pkl", "wb") as f:
    pickle.dump(final_models["7_layers"], f)

INFO:tensorflow:Assets written to: ram://4145d713-d810-484c-b518-b4ae694e4919/assets


In [ ]:
# Dump final results
with open("./model/all_models.pkl", "wb") as f:
    pickle.dump(final_models, f)

INFO:tensorflow:Assets written to: ram://5ccb0e7f-b3f8-4602-9b3c-2ae89d1a2d69/assets
INFO:tensorflow:Assets written to: ram://5d2d95b4-ff82-487d-bd98-ba859e8eced0/assets
INFO:tensorflow:Assets written to: ram://557449b4-6368-4822-a75f-79675a055ab9/assets
INFO:tensorflow:Assets written to: ram://4857368e-b747-43dd-9b2d-6e986317f4b8/assets


In [ ]:
train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((70, 70)),
    torchvision.transforms.RandomCrop((64, 64)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((70, 70)),        
    torchvision.transforms.CenterCrop((64, 64)),            
    torchvision.transforms.ToTensor(),                
    torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.2255))])


train_loader, valid_loader, test_loader = get_dataloaders_cifar10(
    batch_size=BATCH_SIZE,
    validation_fraction=0.1,
    train_transforms=train_transforms,
    test_transforms=test_transforms,
    num_workers=2)

# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    print('Class labels of 10 examples:', labels[:10])
    break

In [ ]:
model = torchvision.models.vgg16(pretrained=True)
model

In [ ]:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
model.classifier[1].requires_grad = True
model.classifier[3].requires_grad = True

In [ ]:
model.classifier[6] = torch.nn.Linear(4096, 10)

In [ ]:
model = model.to(DEVICE)

optimizer = torch.optim.SGD(model.parameters(), momentum=0.9, lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       factor=0.1,
                                                       mode='max',
                                                       verbose=True)

minibatch_loss_list, train_acc_list, valid_acc_list = train_model(
    model=model,
    num_epochs=NUM_EPOCHS,
    train_loader=train_loader,
    valid_loader=valid_loader,
    test_loader=test_loader,
    optimizer=optimizer,
    device=DEVICE,
    scheduler=scheduler,
    scheduler_on='valid_acc',
    logging_interval=100)

plot_training_loss(minibatch_loss_list=minibatch_loss_list,
                   num_epochs=NUM_EPOCHS,
                   iter_per_epoch=len(train_loader),
                   results_dir=None,
                   averaging_iterations=200)
plt.show()

plot_accuracy(train_acc_list=train_acc_list,
              valid_acc_list=valid_acc_list,
              results_dir=None)
plt.ylim([60, 100])
plt.show()